In [15]:


# Step 1: Load the dataset

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('D:\\PROJECTS\\House Price Prediction System\\notebook_and_dataset\\csvdata.csv')

# Step 2: Handle Missing Values
# Example: Replace missing values with the mean of each column
df.fillna('null', inplace=True)

# Step 3: Handle Categorical Variables
# Example: One-hot encode categorical variables
# Assuming 'city' and 'location' are categorical variables
df = pd.get_dummies(df, columns=['City', 'Location'], drop_first=True)

# Step 4: Split the data into features and target variable
X = df.drop('Price', axis=1)  # Features
y = df['Price']               # Target variable

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Feature Scaling (if necessary)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 7: Initialize the XGBoost model
model = XGBRegressor(
    n_estimators=1000,  # number of trees (boosting rounds)
    learning_rate=0.05,  # step size shrinkage to prevent overfitting
    max_depth=5,  # maximum depth of each tree
    subsample=0.7,  # fraction of samples used in each boosting round
    colsample_bytree=0.7,  # fraction of features used in each boosting round
    random_state=42
)

# Step 8: Train the model
model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)

# Step 9: Evaluate the model
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'R-squared (R^2): {r2:.2f}')


Mean Absolute Error (MAE): 7476236.19
Mean Squared Error (MSE): 644447412806204.38
R-squared (R^2): 0.13


In [30]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Example raw dataset
data = pd.read_csv('D:\\PROJECTS\\House Price Prediction System\\notebook_and_dataset\\csvdata.csv')
df = pd.DataFrame(data)

# Split data into features and target variable
X = df.drop('Price', axis=1)
y = df['Price']

# Define categorical and numerical features
categorical_features = ['City', 'Location']
numerical_features = ['Area', 'No. of Bedrooms']

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Append XGBRegressor to preprocessing pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', XGBRegressor())])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Area', 'No. of Bedrooms']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Location'])])),
                ('regressor'...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [39]:
# Predict using the entire test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'R-squared (R^2): {r2:.2f}')

Mean Absolute Error (MAE): 7599306.61
Mean Squared Error (MSE): 644065526075708.62
R-squared (R^2): 0.13


In [41]:
# Predict with raw data
new_data = pd.DataFrame({
    'City': ['Bengaluru'],
    'Area': [600],
    'Location': ['JP Nagara Phase 1'],
    'No. of Bedrooms': [1]
})

predicted_price = model.predict(new_data)

print(f'Predicted Price: {predicted_price[0]:.2f}')


Predicted Price: 6811215.00
